# Count Fasta (Windows-Size, Step-Size)
by _Ken-nosuke Wada_ and _Yoshiko Wada_

`Ver 1.4`<br>
2019-06-14<br>
`Ver 2.0`<br>
2020-11-15<br>
`Ver 2.1`<br>
2023-05-23

## パラメータの設定

In [1]:
dir_fasta     = "fasta"                          # Name of the directory containing the Fasta file
ext_fasta     = [".fasta", ".fas", ".fa", ".f"]  # Fasta file extension
count_letters = ['A', 'C', 'G', 'T']             # List of characters to be included in the frequency count

nuc_length    = 4       # Length of nucleotides to be frequency counted
window_size   = 100000  # Length of segment to count frequency
step_size     = 100000  # Step length when shifting segments and counting
others_th     = 20      # Ignore segments with character content exceeding this value that are not included in the frequency count

trim_rule     = "AAAAAAAAAAA.*$"  # Regular expression matching rules for removing Poly-A tails

In [2]:
import os.path
import glob
import re
from collections import defaultdict

In [3]:
if not os.path.isdir(dir_fasta):
    raise Exception("Fasta files should be included in the directory : " + dir_fasta + ".")

In [4]:
import datetime
start_time = datetime.datetime.now()
print(start_time)

2023-05-23 23:14:12.984571


In [5]:
fasta_files = []
filenames = os.path.join(dir_fasta, '*')

for file in glob.glob(filenames):
    root, ext = os.path.splitext(file)
    if ext in ext_fasta:
        fasta_files.append(file)
        print(file)

if len(fasta_files) == 0:
    raise Exception("Not found Fasta files in the directory : " + dir_fasta + ".")

fasta\Hemi_chr3.fa
fasta\Lepi_chr1-4_22.fa
fasta\Odo_chr10.fa
fasta\Ortho_chr10.fa
fasta\Tri_chr5.fa


In [6]:
class FastaData():
    def __init__(self, filename):
        self.filename = filename
        self.name = os.path.basename(filename)
        self.startDict = {}
        self.dataDict = {}
        self.compListDict = {}
        self.patternListDict = {}
        
        self.readData()
    
    def readData(self):
        with open(self.filename) as f:
            lines = f.readlines()
        
        key = None
        txt = ''
        
        for line in lines:
            if line.startswith('>'):
                if key != None:
                    self.dataDict[key] = txt
                key = line.rstrip('\n')
                txt = ''
            else:
                txt += line.rstrip('\n')
        
        self.dataDict[key] = txt

In [7]:
fasta_data_list = []
debug_print = True

for filename in fasta_files:
    fasta_data = FastaData(filename)
    fasta_data_list.append(fasta_data)
    
    if debug_print:
        print(fasta_data.name)
        print(len(fasta_data.dataDict))
        for key, val in fasta_data.dataDict.items():
            print(key)
            #print(val)
        print('------')

Hemi_chr3.fa
1
>CM037033.1 Schizaphis graminum isolate BZ-2018 chromosome 3, whole genome shotgun sequence
------
Lepi_chr1-4_22.fa
5
>LR990257.1 Aricia agestis genome assembly, chromosome: 1
>LR990258.1 Aricia agestis genome assembly, chromosome: 2
>LR990278.1 Aricia agestis genome assembly, chromosome: 22
>LR990259.1 Aricia agestis genome assembly, chromosome: 3
>LR990260.1 Aricia agestis genome assembly, chromosome: 4
------
Odo_chr10.fa
1
>OV121110.1 Ischnura elegans genome assembly, chromosome: 10
------
Ortho_chr10.fa
1
>CM038243.1 Schistocerca americana isolate TAMUIC-IGC-003095 chromosome 10, whole genome shotgun sequence
------
Tri_chr5.fa
1
>OU830596.1 Limnephilus lunatus genome assembly, chromosome: 5
------


In [8]:
def countPattern(sequence, nuc_length, debug=False):
    sequence = sequence.upper()
    patterns = defaultdict(int)
    
    for i in range(len(sequence) - nuc_length + 1):
        substr = sequence[i:(i + nuc_length)]
        
        stopFlag = False
        for s in substr:
            if s not in count_letters:
                stopFlag = True
                break
        if stopFlag:
            continue
        
        if debug:
            print(substr)
        
        patterns[substr] += 1
    
    return patterns

In [9]:
debug_print = False

patterns_set = set()

for fasta_data in fasta_data_list:
    print(fasta_data.name)
    
    for seq_name in fasta_data.dataDict:
        sequence = fasta_data.dataDict[seq_name]
        print(seq_name)
        
        sequence = sequence.upper()
        
        trimed_seq = re.sub(trim_rule, "", sequence)
        if len(trimed_seq) != len(sequence):
            print("\tTRIM: " + str(len(sequence)) + " -> " + str(len(trimed_seq)))
        
        fasta_data.startDict[seq_name]= []
        fasta_data.patternListDict[seq_name]= []
        fasta_data.compListDict[seq_name] = []
        
        for start in range(0, len(sequence), step_size):
            if start + window_size >= len(sequence):
                break
            
            subsequence = sequence[start:(start + window_size)]
            
            counts = []
            csum = 0
            for x in count_letters:
                num = subsequence.count(x)
                csum += num
                counts.append(num)
            counts.append(len(subsequence) - csum)
            
            others_percent = 100.0 * counts[-1] / len(subsequence)
            if others_percent > others_th:
                print("#" + str(start) + "_" + str(start + window_size - 1))
                print("\tOthers Percent : ", others_percent)
                continue
            
            patterns = countPattern(subsequence, nuc_length)
            
            fasta_data.startDict[seq_name].append(start + 1)
            fasta_data.patternListDict[seq_name].append(patterns)
            fasta_data.compListDict[seq_name].append(counts)
            
            if debug_print:
                print(len(patterns.keys()))
        
            patterns_set |= set(patterns.keys())
 
    if debug_print:
        print('------')

patterns_set = sorted(patterns_set)
nuc_pow = len(count_letters) ** nuc_length

print(str(len(patterns_set)) + " / " + str(nuc_pow))

Hemi_chr3.fa
>CM037033.1 Schizaphis graminum isolate BZ-2018 chromosome 3, whole genome shotgun sequence
	TRIM: 104490323 -> 476
Lepi_chr1-4_22.fa
>LR990257.1 Aricia agestis genome assembly, chromosome: 1
	TRIM: 28198316 -> 8125
>LR990258.1 Aricia agestis genome assembly, chromosome: 2
	TRIM: 24835252 -> 5977
>LR990278.1 Aricia agestis genome assembly, chromosome: 22
	TRIM: 9090845 -> 2486
>LR990259.1 Aricia agestis genome assembly, chromosome: 3
	TRIM: 24265812 -> 2727
>LR990260.1 Aricia agestis genome assembly, chromosome: 4
	TRIM: 22860255 -> 1578
Odo_chr10.fa
>OV121110.1 Ischnura elegans genome assembly, chromosome: 10
	TRIM: 108619349 -> 40780
Ortho_chr10.fa
>CM038243.1 Schistocerca americana isolate TAMUIC-IGC-003095 chromosome 10, whole genome shotgun sequence
	TRIM: 203335778 -> 182423
Tri_chr5.fa
>OU830596.1 Limnephilus lunatus genome assembly, chromosome: 5
	TRIM: 104678855 -> 13199
256 / 256


In [10]:
dirname = "count_" + str(nuc_length) + "_" + str(window_size) + "_" + str(step_size)
dir_count = os.path.join(dir_fasta, dirname)
if not os.path.isdir(dir_count):
    os.mkdir(dir_count)

fname = os.path.join(dir_count, "labels.txt")
labels = ""
with open(fname, "w", encoding="utf-8") as f:
    labels = "\t".join(patterns_set)
    f.write(labels)

count_letters.append('-')

for fasta_data in fasta_data_list:
    fname = os.path.join(dir_count, os.path.splitext(fasta_data.name)[0] + ".cnt")
    print(fname)
    
    with open(fname, "w", encoding="utf-8") as f:
        head  = "%NUCLEOTIDES\t" + str(nuc_length) + "\n"
        head += "%WINDOWSIZE\t" + str(window_size) + "\n"
        head += "%STEPSIZE\t" + str(step_size) + "\n"
        f.write(head)
        
        for seq_name, patternDict in fasta_data.patternListDict.items():
            print(seq_name)
            f.write(seq_name + "\n")
            
            for segmentNo, segmentDict in enumerate(patternDict):
                start = fasta_data.startDict[seq_name][segmentNo]
                end   = start + window_size - 1
                
                compline = "#" + str(start) + "_" + str(end)
                
                counts = fasta_data.compListDict[seq_name][segmentNo]
                                
                for idx, count in enumerate(counts):
                    compline += "\t" + count_letters[idx] + " : " + str(count)
                f.write(compline + "\n")
                
                segmentKeys = segmentDict.keys()
                
                line = []
                for pattern in patterns_set:
                    count = 0
                    if pattern in segmentKeys:
                        count = segmentDict[pattern]
                    line.append(count)
                line = [str(x) for x in line]
                line = "\t".join(line)
                
                f.write(line + "\n")

fasta\count_4_100000_100000\Hemi_chr3.cnt
>CM037033.1 Schizaphis graminum isolate BZ-2018 chromosome 3, whole genome shotgun sequence
fasta\count_4_100000_100000\Lepi_chr1-4_22.cnt
>LR990257.1 Aricia agestis genome assembly, chromosome: 1
>LR990258.1 Aricia agestis genome assembly, chromosome: 2
>LR990278.1 Aricia agestis genome assembly, chromosome: 22
>LR990259.1 Aricia agestis genome assembly, chromosome: 3
>LR990260.1 Aricia agestis genome assembly, chromosome: 4
fasta\count_4_100000_100000\Odo_chr10.cnt
>OV121110.1 Ischnura elegans genome assembly, chromosome: 10
fasta\count_4_100000_100000\Ortho_chr10.cnt
>CM038243.1 Schistocerca americana isolate TAMUIC-IGC-003095 chromosome 10, whole genome shotgun sequence
fasta\count_4_100000_100000\Tri_chr5.cnt
>OU830596.1 Limnephilus lunatus genome assembly, chromosome: 5


In [12]:
end_time = datetime.datetime.now()
print("\nStart   Time  : " + str(start_time))
print(  "End     Time  : " + str(end_time))
print(  "Elapsed Time  : " + str(end_time - start_time) + "\n")


Start   Time  : 2023-05-23 23:14:12.984571
End     Time  : 2023-05-23 23:52:12.465683
Elapsed Time  : 0:37:59.481112

